In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Set up paths to your dataset
train_dir = 'CNN_dataset/training_set'
test_dir = 'CNN_dataset/test_set'

# Image data generators
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                target_size=(150, 150),
                                                batch_size=32,
                                                class_mode='binary')

# Building the CNN model
cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compiling the model
cnn.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Training the model
trained_cnn = cnn.fit(train_generator,
                    epochs=25,
                    validation_data=test_generator)

# Save the model after training
cnn.save('CNN_model/cnn_cat_or_dog_v5.0.keras')

# Evaluating the model
loss, accuracy = cnn.evaluate(test_generator)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the saved model
cnn = tf.keras.models.load_model('CNN_model/cnn_cat_or_dog_v4.0.keras')

# Load and preprocess the image
test_image = 'CNN_dataset/single_prediction/cat_or_dog_7.jpg'
test_image = image.load_img(test_image, target_size=(150, 150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)  # Add batch dimension
test_image /= 255.0  # Normalize the image to the same scale as training data

# Make a prediction
prediction = cnn.predict(test_image)

# Interpret the result
if prediction[0] > 0.5:
    print(f'Dog. \nConfidence: {prediction[0][0]*100:.0f} %')
else:
    print(f'Cat. \nConfidence: {(1 - prediction[0][0])* 100:.0f} %')

In [ ]:
import streamlit as st
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image

# Load the trained model
model = tf.keras.models.load_model('CNN_model/cnn_cat_or_dog_v4.0.keras')

# Function to preprocess image
def preprocess_image(img):
    img = img.resize((150, 150))  # Resize image to match the model's input size
    img_array = image.img_to_array(img)  # Convert the image to an array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the image to the same scale as training data
    return img_array

# Streamlit app
st.title("🐾 Cat or Dog Classifier using CNN")

st.write("Upload an image to classify 🐱 or 🐶. image format has to be jpg, jpeg or png")

# Image upload
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Open and display the uploaded image
    image_file = Image.open(uploaded_file)
    st.image(image_file, caption="Uploaded Image", use_column_width=True)

    # Preprocess the uploaded image
    img_array = preprocess_image(image_file)

    # Make a prediction
    prediction = model.predict(img_array)

    # Interpret the result and display it
    if prediction[0] > 0.5:
        st.write(f"**Prediction:** Dog")
        st.write(f"**Confidence:** {prediction[0][0]*100:.2f}%")
    else:
        st.write(f"**Prediction:** Cat")
        st.write(f"**Confidence:** {(1 - prediction[0][0])*100:.2f}%")
